Revoir minimum WR distances : très long 

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from swiss_rounds_simulator.utils import simulate_tournament, simulate_n_tournaments, compare_settings
from swiss_rounds_simulator.utils import  initiate_league, assign_opponents, play_round
from swiss_rounds_simulator.mwmatching import maxWeightMatching


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [74]:
def rank_table(league_table:pd.DataFrame, n_rounds:int):
    if 'rank' not in league_table.columns:
        league_table['rank'] = [0] * league_table.shape[0]

    init_ranks = league_table['rank']
    multi_given_ranks = init_ranks.value_counts()[init_ranks.value_counts()>1].index
    if multi_given_ranks.shape[0] > 0:
        keep_ranking = True
    else :
        keep_ranking = False

    while keep_ranking:
        for initial_rank in multi_given_ranks :
            subcomp_df = league_table.query(f"rank =={initial_rank}")
            subcomps = []
            for score in subcomp_df['Nb_win'].unique():
                subtable = subcomp_df.query(f"Nb_win == {score}")
                for team in subtable.index:
                    team_subcomp_games = []
                    for i in range(n_rounds):
                        opponent = subcomp_df.loc[team,f'R{i+1}_opponent']
                        if opponent in subtable.index:
                            result = subcomp_df.loc[team,f'R{i+1}_result']
                            team_subcomp_games.append(1 if result == 'Win' else 0)
                    team_subscore = np.mean(team_subcomp_games) if len(team_subcomp_games) >0 else 0.5
                    subcomps.append({'team':team, 'subscore':team_subscore})
            tmp_df = pd.DataFrame(subcomps).set_index('team')
            subcomp_df = subcomp_df.merge(tmp_df, left_index=True, right_index=True)
            subcomp_df['tmp_rank'] = 1000000 * subcomp_df['Nb_win'] + 1000 * subcomp_df['subscore'] + subcomp_df['OWR']
            r_df = subcomp_df['tmp_rank'].sort_values(ascending=False).drop_duplicates().reset_index().drop(columns = 'index').reset_index().set_index('tmp_rank')
            
            for team in subcomp_df.index:
                league_table.loc[team,'rank'] = r_df.loc[subcomp_df.loc[team,'tmp_rank'],'index'] + initial_rank
        
        final_ranks = league_table['rank']
        if final_ranks.equals(init_ranks):
            keep_ranking = False
        else :
            init_ranks = final_ranks    

    league_table.sort_values(by='rank', inplace = True)
    league_table['rank'] = league_table['rank'] +1
    return league_table


In [75]:
n_teams = 16
n_rounds = 6

lt = simulate_tournament(nb_teams=n_teams, nb_games=n_rounds, method='probabilistic', delta_level='linear', verbose = False)
lt = rank_table(lt,n_rounds)
lt

,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank
b,1,0.9375,[],5,1,0.833333,0.527778,p,Win,e,Loss,o,Win,i,Win,d,Win,h,Win,1
f,5,0.6875,[],5,1,0.833333,0.5,i,Loss,m,Win,c,Win,a,Win,h,Win,e,Win,2
h,7,0.5625,[],4,2,0.666667,0.666667,g,Win,c,Win,d,Win,e,Win,f,Loss,b,Loss,3
a,0,1.0,[],4,2,0.666667,0.472222,o,Win,l,Loss,k,Win,f,Loss,p,Win,i,Win,4
e,4,0.75,[],4,2,0.666667,0.583333,k,Win,b,Win,l,Win,h,Loss,j,Win,f,Loss,5
d,3,0.8125,[],4,2,0.666667,0.527778,n,Win,i,Win,h,Loss,l,Win,b,Loss,k,Win,6
k,10,0.375,[],3,3,0.5,0.527778,e,Loss,n,Win,a,Loss,p,Win,c,Win,d,Loss,7
i,8,0.5,[],3,3,0.5,0.611111,f,Win,d,Loss,p,Win,b,Loss,l,Win,a,Loss,8
g,6,0.625,[],3,3,0.5,0.333333,h,Loss,p,Loss,n,Loss,m,Win,o,Win,j,Win,9
c,2,0.875,[],3,3,0.5,0.444444,m,Win,h,Loss,f,Loss,o,Win,k,Loss,n,Win,10


In [78]:
lt.query("Nb_win == 2")


,Id,Level,Strategy,Nb_win,Nb_loss,Win_rate,OWR,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result,R6_opponent,R6_result,rank
p,15,0.0625,[],2,4,0.333333,0.555556,b,Loss,g,Win,i,Loss,k,Loss,a,Loss,l,Win,11
o,14,0.125,[],2,4,0.333333,0.472222,a,Loss,j,Win,b,Loss,c,Loss,g,Loss,m,Win,12
l,11,0.3125,[],2,4,0.333333,0.527778,j,Win,a,Win,e,Loss,d,Loss,i,Loss,p,Loss,13
j,9,0.4375,[],2,4,0.333333,0.361111,l,Loss,o,Loss,m,Win,n,Win,e,Loss,g,Loss,14
n,13,0.1875,[],2,4,0.333333,0.416667,d,Loss,k,Loss,g,Win,j,Loss,m,Win,c,Loss,15


In [66]:
init_ranks

a    0
c    1
d    2
g    3
h    4
e    5
b    6
f    7
Name: rank, dtype: int64

In [22]:
comp = False
if comp :
    compare_settings(n_tournaments=100, 
                 n_teams=36, 
                 n_rounds = 8, 
                 delta_level ='linear',
                 strategies={'i':[1]}, 
                 probabilistic=True, 
                 deterministic=True)
